# Neural Network for Music Speech Separation

In [1]:
import tensorflow as tf

2022-08-04 17:10:39.423573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-04 17:10:39.423639: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from tensorflow import keras

In [3]:
import soundfile as sf
import librosa as lb
import numpy as np
from matplotlib import pyplot as plt

In [4]:
import os

## Reading DataSet

In [5]:
path = "/home/rajesh/MusicSpeechSeparartion/Dataset/"

folders = os.listdir(path)
folders

['Overlay', 'Mixture', 'speech', 'music']

In [6]:
audiofiles, samplerates = [], []
for folder in folders:
    files = os.listdir(path+'/'+folder)
    files = sorted(files)
    audio, sr = [], []
    for file in files:
        data, samplerate = sf.read(path+'/'+folder+'/'+file)
        audio.append(data)
        sr.append(samplerate)
    audiofiles.append(audio)
    samplerates.append(sr)

In [7]:
overlay = np.array(audiofiles[0])
mixture = np.array(audiofiles[1])
speech = np.array(audiofiles[2])
music = np.array(audiofiles[3])

dataset = [overlay, mixture, speech, music]

### Short Time Fourier Transform

In [8]:
datasetstft = []
for data in dataset:
    spect = []
    for audio in data:
        stft = np.abs(lb.stft(audio))
        spect.append(stft)
    datasetstft.append(spect)

In [9]:
overlay_stft = np.array(datasetstft[0])
mixture_stft = np.array(datasetstft[1])
speech_stft = np.array(datasetstft[2])
music_stft = np.array(datasetstft[3])

dataset_stft = [overlay_stft, mixture_stft, speech_stft, music_stft]

In [10]:
mixture_stft.shape, speech_stft.shape

((50, 1025, 1292), (50, 1025, 646))

In [11]:
groundtruth_stft = []
for i in range(50):
    groundtruth_stft.append(np.hstack((speech_stft[i], music_stft[i])))

    
groundtruth_stft = np.array(groundtruth_stft)
groundtruth_stft.shape

(50, 1025, 1292)

## DNN Model

In [12]:
from keras import layers

In [13]:
input_dim = (1025, 1292)
output_dim = 1025*1292

inputs = layers.Input(shape=input_dim)
x = layers.Flatten()(inputs)

x = layers.Dense(units=128, activation='relu')(x)
x = layers.BatchNormalization()(x)

x = layers.Dense(units=128, activation='relu')(x)
x = layers.BatchNormalization()(x)

outputs = layers.Dense(units=output_dim, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

2022-08-04 17:11:07.991431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-04 17:11:07.991457: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-04 17:11:07.991474: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Rajesh): /proc/driver/nvidia/version does not exist
2022-08-04 17:11:07.991619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1025, 1292)]      0         
                                                                 
 flatten (Flatten)           (None, 1324300)           0         
                                                                 
 dense (Dense)               (None, 128)               169510528 
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                             

In [15]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)

In [16]:
X_train = mixture_stft
y_train = groundtruth_stft

X_train.shape, y_train.shape

((50, 1025, 1292), (50, 1025, 1292))

In [22]:
model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=2,
    validation_split=0.1,
    verbose=1
)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/rajesh/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/rajesh/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/rajesh/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/rajesh/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.